In [ ]:
import pandas as pd
import motmetrics as mm
import numpy as np
import matplotlib.pyplot as plt

tracking_output = pd.read_csv('code_output_data_microns.csv')
ground_truth = pd.read_csv('consensus_ground_truth_microns.csv')

# Ground truth and predicted data as DataFrames
gt_df = pd.DataFrame({'FrameId': ground_truth['frame'], 'Id': ground_truth['particle'], 'X': ground_truth['X'], 'Y': ground_truth['Y']})
pred_df = pd.DataFrame({'FrameId': tracking_output['time'], 'Id': tracking_output['track_id'], 'X': tracking_output['X'], 'Y': tracking_output['Y']})

# Initialize accumulator for metrics
acc = mm.MOTAccumulator(auto_id=True)
for frame in sorted(set(gt_df['FrameId']).union(pred_df['FrameId'])):
    gt_positions = gt_df[gt_df['FrameId'] == frame][['X', 'Y']].values
    pred_positions = pred_df[pred_df['FrameId'] == frame][['X', 'Y']].values
    acc.update(gt_df[gt_df['FrameId'] == frame]['Id'].values,
               pred_df[pred_df['FrameId'] == frame]['Id'].values,
               mm.distances.norm2squared_matrix(gt_positions, pred_positions, max_d2=10000))

# Compute metrics
metrics = ['idf1', 'idp', 'idr', 'recall', 'precision', 'num_false_positives', 'num_misses', 'num_switches', 'mota', 'motp']
summary = mm.metrics.create().compute(acc, metrics=metrics, name='Overall')
print(summary)

# Compute IoU (Intersection over Union)
iou_scores = []
tracked_ids = set(pred_df['Id'])
ground_truth_ids = set(gt_df['Id'])
for obj_id in tracked_ids & ground_truth_ids:
    track_tracked = set(pred_df[pred_df['Id'] == obj_id]['FrameId'])
    track_gt = set(gt_df[gt_df['Id'] == obj_id]['FrameId'])
    intersection = len(track_tracked & track_gt)
    union = len(track_tracked | track_gt)
    iou_scores.append(intersection / union if union > 0 else 0)
mean_iou = np.mean(iou_scores) if iou_scores else None
print(f"Mean IoU: {mean_iou:.4f}" if mean_iou else "No overlapping tracks for IoU calculation.")

# Visualize trajectories
plt.figure(figsize=(10, 6))
for obj_id in gt_df['Id'].unique():
    plt.plot(gt_df[gt_df['Id'] == obj_id]['X'], gt_df[gt_df['Id'] == obj_id]['Y'], label=f"GT {obj_id}")
for obj_id in pred_df['Id'].unique():
    plt.plot(pred_df[pred_df['Id'] == obj_id]['X'], pred_df[pred_df['Id'] == obj_id]['Y'], linestyle='--', label=f"Pred {obj_id}")
plt.title('Ground Truth vs Predicted Trajectories')
plt.xlabel('X'); plt.ylabel('Y')
plt.legend(); plt.grid(True); plt.gca().invert_yaxis()
plt.show()
